In [1]:
import urllib
import os.path
import pathlib
import pandas as pd
import numpy as np
import functools
import itertools

from nameparser import HumanName
import humanize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 250)

In [2]:
if not os.path.isfile("2021_Contributions.csv"):
    url = "https://www.nyccfb.info/DataLibrary/2021_Contributions.csv"
    urllib.request.urlretrieve(url, "2021_Contributions.csv")
    
if not os.path.isfile("eLobbyist_data.csv"):
    url = "https://data.cityofnewyork.us/api/views/fmf3-knd8/rows.csv?accessType=DOWNLOAD"
    urllib.request.urlretrieve(url, "eLobbyist_data.csv")
    

In [3]:
contributions = pd.read_csv("2021_Contributions.csv", header=0, dtype={ 'NAME': str, 'STRNO': str, 'STRNAME': str, 'APARTMENT': str , 'INTZIP': str, 'EXEMPTCD': str })

In [4]:
def employer_format(s):
    if not isinstance(s, str):
        return s
    if s.lower() == 'not employed' or s.lower() == 'unemployed':
        return 'Not Employed'
    elif  s.lower() in ['self-employed', 'self employed', 'self', 'Self Employee']:
        return 'Self Employed'
    elif s.lower() == 'none':
        return 'None'
    elif s.lower() in ['homemaker', 'housewife']:
        return 'Homemaker'
    elif s.lower() in ['Citigroup', 'citi']:
        return 'Citigroup'
    elif 'paul weiss' in s.lower():
        return 'Paul Weiss'
    elif ('Skadden Arps' in s.lower() or s == 'Skadden'):
        return 'Skadden Arps'
    elif 'Rosen & Katz' in s or s == 'Wachtell Lipton':
        return 'Wachtell, Lipton, Rosen & Katz'
    elif 'A&E Real Estate' in s.lower():
        return 'A&E Real Estate'
    else:
        return s


contributions['EMPNAME'] = contributions['EMPNAME'].apply(employer_format)

In [5]:
contributions

,ELECTION,OFFICECD,RECIPID,CANCLASS,RECIPNAME,COMMITTEE,FILING,SCHEDULE,PAGENO,SEQUENCENO,REFNO,DATE,REFUNDDATE,NAME,C_CODE,STRNO,STRNAME,APARTMENT,BOROUGHCD,CITY,STATE,ZIP,OCCUPATION,EMPNAME,EMPSTRNO,EMPSTRNAME,EMPCITY,EMPSTATE,AMNT,MATCHAMNT,PREVAMNT,PAY_METHOD,INTERMNO,INTERMNAME,INTSTRNO,INTSTRNM,INTAPTNO,INTCITY,INTST,INTZIP,INTEMPNAME,INTEMPSTNO,INTEMPSTNM,INTEMPCITY,INTEMPST,INTOCCUPA,PURPOSECD,EXEMPTCD,ADJTYPECD,RR_IND,SEG_IND,INT_C_CODE
0,2021,1,2563,P,"Tirschwell, Sara A",H,7,ABC,NaN,NaN,R0000209,3/6/2021,NaN,"""Dombrowski,"", Ray",IND,NaN,NaN,NaN,NaN,Haverford,PA,19041,Turnaround Manager,Alvarez & Marsal,NaN,600 Madison Avenue,New York,NY,2000.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
1,2021,55,2345,P,"Camarena, Rodrigo G",H,6,ABC,NaN,NaN,R0001664,12/17/2020,NaN,"(Dougherty) LoBianco, Kelly",IND,NaN,NaN,NaN,Z,Los Angeles,CA,90068,Chief Program Officer,The HOPE Program,1,Smith Street,Brooklyn,NY,38.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
2,2021,55,2414,P,"Low, Jenny L",H,6,ABC,NaN,NaN,R0000080,9/5/2020,NaN,"(Eng) Lawton, Linnit",IND,NaN,NaN,NaN,K,Brooklyn,NY,11201,Human Resources,Hamilton Madison House,253,South Street,New York,NY,25.0,25,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
3,2021,55,283,P,"Gennaro, James F",M,7,ABC,NaN,NaN,R0002295,3/3/2021,NaN,"(Orlow) Husarsky, Miriam",IND,NaN,NaN,NaN,Q,Richmond Hill,NY,11418,Speech Therapist,NYC Dept of Education,63-55,102nd street,Rego Park,NY,175.0,175,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
4,2021,55,2454,P,"Boghosian Murphy, Leslie",H,6,ABC,NaN,NaN,R0000171,10/19/2020,NaN,"(Zimmerman)Shaich, Carol",IND,NaN,NaN,NaN,M,Ny,NY,10027,Retired,NaN,NaN,NaN,NaN,NaN,25.0,25,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369620,2021,11,2546,P,"Yang, Andrew",H,8,ABC,NaN,NaN,R0079799,5/13/2021,NaN,"Zymaris, Mai",IND,NaN,NaN,NaN,Z,BRAINTREE,MA,2184,Attorney,Morrison & Foerster,428,John Mahar Hwy Unit 207,BRAINTREE,MA,50.0,0,275.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
369621,2021,11,2393,P,"Wiley, Maya D",H,7,ABC,NaN,NaN,R0034934,2/13/2021,NaN,"Zysermann, Noemie",IND,NaN,NaN,NaN,M,New York,NY,10023,Consultant,Self Employed,266,W 73rd st,New York,NY,25.0,25,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
369622,2021,33,1559,P,"Johnson, Corey D",L,3,ABC,NaN,NaN,R0016683,7/11/2019,NaN,"Zyskowska, Celina",IND,NaN,NaN,NaN,Q,Kew Gardens,NY,11415,Accounting,Commodore Construction,602,South 3rd Avenue,Mt Vernon,NY,250.0,250,0.0,2,16.0,"Oneill, Brendan",NaN,NaN,NaN,Long Island City,NY,11104,Commodore Construction,602,South 3rd Avenue,Mt Vernon,NY,Labor Manager,NaN,NaN,NaN,N,N,IND
369623,2021,11,1545,P,"Adams, Eric L",L,1,ABC,NaN,NaN,R0003574,6/12/2018,NaN,"Zyskowski, Joseph",IND,NaN,NaN,NaN,Q,Middle Village,NY,11379,Owner,K Construction,65-58,Admiral Avenue,Middle Village,NY,5000.0,250,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN


In [6]:
major_candidates = [
    'Adams, Eric L',
    'Donovan, Shaun',
    'Garcia, Kathryn A',
    'McGuire, Raymond J',
    'Morales, Dianne',
    'Stringer, Scott M',
    'Wiley, Maya D',
    'Yang, Andrew'
]

mayor_candidates_names = contributions[contributions['OFFICECD'].isin(['1', '11'])]['RECIPNAME'].unique()
candidate_last_names = dict(zip(mayor_candidates_names, map(lambda s: HumanName(s).last, mayor_candidates_names)))
candidate_last_names.values()

dict_values(['Tirschwell', 'Wiley', 'Yang', 'Morales', 'McGuire', 'Stringer', 'Sliwa', 'Cullen', 'Garcia', 'Donovan', 'Menchaca', 'Adams', 'Mateo', 'Diaz', 'Pepitone', 'Taylor', 'AbdulMalik', 'Chang', 'Sutton', 'Rose', 'Prince', 'Wright', 'Guimaraes', 'Flores', 'Emilien', 'Foldenauer', 'Filipchenko', 'Laurel-Smith', 'Kavovit', 'Francis', 'Oremland', 'Nunez', 'Downs', 'Kaplan', 'Miles', 'Reaves', 'Coenen', 'Bunea', 'Pinto', 'Fitzgerald', 'Krietchman', "O'Hagan", 'Seidman', 'Seely'])

In [7]:
mayor_donations = contributions[contributions["RECIPNAME"].isin(major_candidates) & contributions["SCHEDULE"].eq("ABC")].copy()

mayor_donations.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125557 entries, 38 to 369623
Data columns (total 52 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ELECTION    125557 non-null  int64  
 1   OFFICECD    125557 non-null  object 
 2   RECIPID     125557 non-null  object 
 3   CANCLASS    125557 non-null  object 
 4   RECIPNAME   125557 non-null  object 
 5   COMMITTEE   125557 non-null  object 
 6   FILING      125557 non-null  int64  
 7   SCHEDULE    125557 non-null  object 
 8   PAGENO      0 non-null       float64
 9   SEQUENCENO  0 non-null       float64
 10  REFNO       125557 non-null  object 
 11  DATE        125557 non-null  object 
 12  REFUNDDATE  0 non-null       object 
 13  NAME        125557 non-null  object 
 14  C_CODE      125557 non-null  object 
 15  STRNO       0 non-null       object 
 16  STRNAME     0 non-null       object 
 17  APARTMENT   0 non-null       object 
 18  BOROUGHCD   125443 non-null  object 
 19  C

In [8]:
mayor_donations.NAME.value_counts()

Jordan, Tyron        63
Cole, Sarah          59
Perkins, D           58
Warren, Roxanne      56
Koo, Julia           56
                     ..
Griffiths, Roland     1
Munn, Monica          1
Tang, Alex            1
McLean, Doug          1
Margolin, Susan       1
Name: NAME, Length: 74907, dtype: int64

In [9]:
mayor_donations['C_CODE'].unique()

array(['PCOMP', 'CORP', 'IND', 'CAN', 'EMPO', 'FAM', 'OTHR', 'PCOMC',
       'PCOMZ', 'LLC'], dtype=object)

In [10]:
def to_money(i):
     return "$" + humanize.intcomma(round(i))

def to_person_name(s):
    return str(HumanName(s))

def parse_name(s):
    names = []
    
    human_name = HumanName(s)
    
    names.append(f"{human_name.last}, {human_name.first}")
    
    if human_name.suffix:
        names.append(f"{human_name.last} {human_name.suffix}, {human_name.first}")
    
    if human_name.middle:
        names.append(f"{human_name.last}, {human_name.first} {human_name.middle}")
    
        if len(human_name.middle) > 1:
            names.append(f"{human_name.last}, {human_name.first} {human_name.middle[0]}")
    
    return names


def name_variations_from_file(file):
    variations = []
    
    with open(file, 'r') as f:
        for line in f:
            for variation in parse_name(line):
                variations.append(variation.upper())

    return variations
                
real_estate_names_from_littlesis = name_variations_from_file('./real_estate_names.txt')
worst_landlords = name_variations_from_file('./worst_landlords.txt')

In [11]:
mayor_donations['realestate_littlesis'] = mayor_donations['NAME'].str.upper().isin(real_estate_names_from_littlesis)
mayor_donations['worst_landlord'] = mayor_donations['NAME'].str.upper().isin(worst_landlords)
mayor_donations['works_in_realestate'] = mayor_donations['OCCUPATION'].str.contains(r'(?i).*REAL ESTATE.*')
mayor_donations['does_lobbying'] = mayor_donations['OCCUPATION'].str.upper().eq('LOBBYIST')

In [12]:
flagged_donations = mayor_donations[ (mayor_donations['realestate_littlesis'] | mayor_donations['worst_landlord'] |  mayor_donations['works_in_realestate']  | mayor_donations['does_lobbying'] | ( mayor_donations['C_CODE'] == "LLC"))]
flagged_donations_major_candidates = flagged_donations[flagged_donations["RECIPNAME"].isin(major_candidates)]

columns = ['ELECTION', 'RECIPID', 'RECIPNAME', 'COMMITTEE', 'NAME', 'AMNT', 'DATE', 'REFUNDDATE', 'CITY', 'STATE', 'ZIP', 'OCCUPATION', 'EMPNAME', 'C_CODE', 'realestate_littlesis', 'worst_landlord', 'works_in_realestate']

flagged_donations.sort_values(['RECIPNAME', 'DATE'])[columns]

,ELECTION,RECIPID,RECIPNAME,COMMITTEE,NAME,AMNT,DATE,REFUNDDATE,CITY,STATE,ZIP,OCCUPATION,EMPNAME,C_CODE,realestate_littlesis,worst_landlord,works_in_realestate
63374,2021,1545,"Adams, Eric L",L,"Cohen, Adir",5100.0,1/10/2019,NaN,Brooklyn,NY,11230,Real Estate,Renaissance Realty Group,IND,False,False,True
124630,2021,1545,"Adams, Eric L",L,"Gottlieb, Israel",1000.0,1/10/2019,NaN,Lakewood Township,NJ,8701,Real estate investor,Self Employed,IND,False,False,True
212793,2021,1545,"Adams, Eric L",L,"Matheus, Donald",500.0,1/10/2019,NaN,Brooklyn,NY,11238,Real estate,Albatross group,IND,False,False,True
321999,2021,1545,"Adams, Eric L",L,"Strulovitch, Baila",2000.0,1/10/2019,NaN,Brooklyn,NY,11205,Real Estate,Self Employee,IND,False,False,True
192053,2021,1545,"Adams, Eric L",L,"Levy, Danny",100.0,1/10/2020,NaN,Brooklyn,NY,11230,Real estate,Self Employed,IND,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346745,2021,2546,"Yang, Andrew",H,"Wang, Kevin",100.0,5/6/2021,NaN,Somerville,MA,2143,Software Engineer,Wellframe,IND,True,False,False
146638,2021,2546,"Yang, Andrew",H,"Holt, Giles",50.0,5/7/2021,NaN,New York,NY,10065,Real Estate,KBD Holdings,IND,False,False,True
188504,2021,2546,"Yang, Andrew",H,"Lee, Saehan",50.0,5/7/2021,NaN,Santa Monica,CA,90401,Manager Real Estate,Knowledge Universe,IND,False,False,True
103383,2021,2546,"Yang, Andrew",H,"Fishkind, Jason",2000.0,5/8/2021,NaN,Purchase,NY,10577,Real Estate,OOHH,IND,False,False,True


In [13]:
flagged_donations.sort_values(['RECIPNAME', 'DATE']).to_csv('flagged_donations.csv')

In [14]:
total_amount_recieved = mayor_donations[["RECIPNAME", "AMNT"]].groupby("RECIPNAME").sum().sort_values(by="AMNT", ascending=False)
total_amount_recieved["AMNT"]  = total_amount_recieved["AMNT"].apply(to_money)
total_amount_recieved = total_amount_recieved.rename_axis('Candidates').rename(columns={'AMNT': 'Amount'})
total_amount_recieved

,Amount
Candidates,
"McGuire, Raymond J","$10,093,818"
"Adams, Eric L","$4,739,061"
"Yang, Andrew","$3,600,747"
"Donovan, Shaun","$2,858,059"
"Stringer, Scott M","$2,839,137"
"Wiley, Maya D","$1,598,117"
"Garcia, Kathryn A","$1,242,811"
"Morales, Dianne","$860,549"


In [15]:
def largest_donors_per_candidate(donations, limit=15):
    """Dataframe of largest donors for each candidate"""
    agg = donations[["RECIPNAME", "AMNT", "NAME", "EMPNAME"]].groupby(["RECIPNAME", "NAME"]).agg({'AMNT': sum, 'EMPNAME': lambda s: s.unique() }) # .sort_values(by='AMNT', ascending=False) # .groupby(level=0, group_keys=False).head(20) # .reset_index() # ort_values(by='AMNT')  # .groupby(level=0, group_keys=False).apply(lambda x: x.sort_values(('AMNT', 'sum'), ascending=False))
        
    return agg['AMNT'].groupby('RECIPNAME', group_keys=False).apply(lambda x: x.sort_values(ascending=False).head(limit).apply(to_money)).to_frame()
    

In [16]:
largest_donors_per_candidate(mayor_donations, limit=30)

AMNT
RECIPNAME          NAME                                     
Adams, Eric L      Bensusan, Danny                   $12,100
                   Waisman, Shai                     $10,200
                   Guzel, Murat                      $10,100
                   Singh, Navdeep                    $10,000
                   Serova, Daria                     $10,000
                   Boutross, Diana                   $10,000
                   Bartlett, Gregory                  $7,600
                   Karandikar, Satish                 $7,100
                   Akcetin, Mehmet                    $7,000
                   David, Emanuel                     $6,600
                   Cerchione, Gregory                 $6,000
                   Razi, Sami Uddin                   $6,000
                   Gravante, Nicholas                 $5,750
                   Basusta, Yusuf                     $5,500
                   Daneshvar, Michelle                $5,350
                   Marino, John                       $5,250
                   Fuzaylova, Michael                 $5,100
                   Dalamal, Aroon                     $5,100
                   Barricelli, Louis                  $5,100
                   Hraki, O                           $5,100
                   Belt, Antonia                      $5,100
                   Gaudet, Anne M                     $5,100
                   Renaud, Stephen                    $5,100
                   georgiton, tony                    $5,100
                   Perez San Martin, Carlos           $5,100
                   glazer, jaf                        $5,100
                   Basaranlar, Tumay                  $5,100
                   Shamis, Alina                      $5,100
                   Perdomo, Jessica                   $5,100
                   Kosokhar, Ronen                    $5,100
Donovan, Shaun     Donovan, Shaun                    $19,100
                   Blumenstein, Ruth                 $15,300
                   Zabar, Eli                        $10,200
                   Fredericks, Devon                 $10,200
                   Bingham, Clara                    $10,200
                   Nolen, Eliot                      $10,200
                   Briganti, Stephen                 $10,200
                   Rockefeller, David                $10,200
                   Goodman, Sara                     $10,150
                   Niejelow, Alexander                $8,100
                   Hickenlooper, Robin                $8,000
                   Whang, John                        $6,100
                   Meyer, Audrey                      $5,600
                   Werner, Zachary                    $5,600
                   Donovan, Lucas                     $5,450
                   Wiese, Megan                       $5,450
                   Birch, Eugenie                     $5,350
                   Heilbut, Wilfred                   $5,200
                   Istel, Yves                        $5,200
                   Barber, David                      $5,150
                   Traber, Catherine                  $5,100
                   Wenig, Devin                       $5,100
                   Goldstein, Gordon                  $5,100
                   Begley, Michael                    $5,100
                   Sterne, James                      $5,100
                   Moore, John                        $5,100
                   Goldstein, Barbara                 $5,100
                   Goldrich, Robert                   $5,100
                   Trulson, Derek                     $5,100
                   McBride, Jonathan                  $5,100
Garcia, Kathryn A  Kramer, Mchael                     $2,500
                   Smith, Daniel                      $2,500
                   Lee, Michael                       $2,250
                   Sutton, Loree                      $2,166
                   Ascher, Kat

In [17]:
# Largest Real Estate Donors
largest_donors_per_candidate(flagged_donations_major_candidates, limit=30)

AMNT
RECIPNAME          NAME                              
Adams, Eric L      Boutross, Diana            $10,000
                   tamir, Elliot               $5,100
                   Verrone, Robert             $5,100
                   Berman, Nathan              $5,100
                   Cohen, Adir                 $5,100
                   Borrok, Charles R.          $5,100
                   Tawil, Eliot                $5,100
                   Demirjian, Alexis           $5,100
                   Schwartz, Yoel              $5,000
                   Corman, Jeffrey             $5,000
                   Landau, Solomon             $5,000
                   Shurka, Manny               $5,000
                   Neamonitakis, Michael       $4,000
                   popkin, drew                $4,000
                   Vitucci Sr, Dominick        $3,400
                   Ward, Jesse                 $3,400
                   Fisher, Avi                 $3,400
                   Gindi, jeffrey              $3,175
                   Anagnost, Dick              $3,000
                   Levkov, Viacheslav          $3,000
                   Silviano, John              $3,000
                   Marchi, Dean                $2,500
                   Sciortino, Anthony          $2,500
                   Iannucci, Robert            $2,500
                   Chassin, Jonathan           $2,500
                   Ewers, Sonia                $2,500
                   Okada, Chris                $2,250
                   Basile, Rocco               $2,250
                   Ginoi, Eddie                $2,000
                   Fisher, Kenneth             $2,000
Donovan, Shaun     KATZ, JEFFREY               $5,100
                   Clark, Ric                  $5,100
                   Milstein, Constance J       $5,100
                   Moore, John                 $5,100
                   Friedman, Eileen            $5,100
                   Greenberg, Mark             $5,100
                   Fisher, Winston             $5,100
                   Fish, John F                $5,100
                   Winn, Arthur                $5,100
                   Trulson, Derek              $5,100
                   Elbaum, Abigail Black       $5,100
                   Zeiler, John                $5,100
                   DeBode, Gary                $5,100
                   olshan, andrea              $5,100
                   Stacom, Darcy               $5,100
                   Bernstein, Joshua           $5,100
                   Silverstein, Larry          $5,100
                   Bahar, Aaron                $5,100
                   Silvers, Nicholas           $5,100
                   Brodsky, Daniel             $5,100
                   Sheller, Chris              $5,100
                   Burch, Melissa              $5,075
                   Lake, Robert                $3,500
                   Martin, Rob                 $3,000
                   Coatsworth, Anna            $2,500
                   kuriloff, josh              $2,500
                   Benenson, Lawrence          $2,500
                   Rubler, Neil                $2,500
                   Wiener, Alan                $2,250
                   Kerr, Zachary               $2,000
Garcia, Kathryn A  Durst, Helena               $2,000
                   Kerr, Zachary               $2,000
                   Mosler, Wendy               $2,000
                   MacArthur, Andrew           $2,000
                   Field, Michael              $2,000
                   Kalikow, Peter              $2,000
                   Koeppel, David J            $2,000
                   Yatrakis, Demetrios         $1,500
                   Ezra, Brian                 $1,400
                   Sheehan, Timothy            $1,300
                   Thypin, Benjamin            $1,250
                   Wark, Jesse                 $1,050
                   Ghossein, Yara              $1,000

In [18]:
def cfb_link(name, only_mayor=False):
    # url = "https://www.nyccfb.info/FTMSearch/Candidates/Contributions?ec=2021&rt=can&ir={}"
    
    if only_mayor:
        url = "https://www.nyccfb.info/FTMSearch/Candidates/Contributions?ec=2021&rt=can&ofc=1%2C11&ir={}"
    else:
        url = "https://www.nyccfb.info/FTMSearch/Candidates/Contributions?ec=2021&rt=can&ir={}"
        

    return url.format(urllib.parse.quote_plus(name))


def all_top_donors_for(candidate):
        columns = ['RECIPNAME', 'DATE', 'AMNT', 'NAME', 'ZIP', 'OCCUPATION', 'EMPNAME']
        donations = mayor_donations[mayor_donations['RECIPNAME'] == candidate][columns]
        donations["name"] =  donations["NAME"].apply(to_person_name)
        donations.rename(columns={'NAME': 'cfb_name', 'OCCUPATION': 'cfb_occupation', 'EMPNAME': 'cfb_employer'}, inplace=True)
        donations['cfb_link'] = donations['cfb_name'].apply(cfb_link)
        return donations.sort_values('AMNT', ascending=False)

def real_estate_donors_for(candidate):
        columns = ['RECIPNAME', 'DATE', 'AMNT', 'NAME', 'ZIP', 'OCCUPATION', 'EMPNAME']
        donations = flagged_donations_major_candidates[flagged_donations_major_candidates['RECIPNAME'] == candidate][columns]
        donations["name"] =  donations["NAME"].apply(to_person_name)
        donations.rename(columns={'NAME': 'cfb_name', 'OCCUPATION': 'cfb_occupation', 'EMPNAME': 'cfb_employer'}, inplace=True)
        donations['cfb_link'] = donations['cfb_name'].apply(functools.partial(cfb_link, only_mayor=True))
        return donations.sort_values('AMNT', ascending=False)

In [19]:
real_estate_donors_for('Donovan, Shaun').head(10)

,RECIPNAME,DATE,AMNT,cfb_name,ZIP,cfb_occupation,cfb_employer,name,cfb_link
366523,"Donovan, Shaun",1/4/2021,5100.0,"Zeiler, John",10023,Real Estate,Hudson Housing Capital,John Zeiler,https://www.nyccfb.info/FTMSearch/Candidates/C...
61828,"Donovan, Shaun",2/21/2021,5100.0,"Clark, Ric",11201,Managing Partner,WatermanClark,Ric Clark,https://www.nyccfb.info/FTMSearch/Candidates/C...
229420,"Donovan, Shaun",3/6/2021,5100.0,"Moore, John",35242,Real Estate Finance/Dev,Highland Commercial Mortgage/A,John Moore,https://www.nyccfb.info/FTMSearch/Candidates/C...
225637,"Donovan, Shaun",4/12/2021,5100.0,"Milstein, Constance J",10014,Real Estate Owner/Principal,"Ogden CAP Properties, LLC",Constance J Milstein,https://www.nyccfb.info/FTMSearch/Candidates/C...
167795,"Donovan, Shaun",1/28/2021,5100.0,"KATZ, JEFFREY",10023,Real Estate Developer,Sherwood Equities Inc.,JEFFREY KATZ,https://www.nyccfb.info/FTMSearch/Candidates/C...
307663,"Donovan, Shaun",3/5/2021,5100.0,"Silvers, Nicholas",10007,Real Estate Developer,Tavros,Nicholas Silvers,https://www.nyccfb.info/FTMSearch/Candidates/C...
307713,"Donovan, Shaun",12/7/2020,5100.0,"Silverstein, Larry",10007,Chairman,Silverstein Properties,Larry Silverstein,https://www.nyccfb.info/FTMSearch/Candidates/C...
127122,"Donovan, Shaun",4/9/2021,5100.0,"Greenberg, Mark",6759,Real Estate,MGRE Co LLC,Mark Greenberg,https://www.nyccfb.info/FTMSearch/Candidates/C...
317587,"Donovan, Shaun",1/29/2021,5100.0,"Stacom, Darcy",10022,Broker,CBRE,Darcy Stacom,https://www.nyccfb.info/FTMSearch/Candidates/C...
335519,"Donovan, Shaun",4/9/2021,5100.0,"Trulson, Derek",10128,Commercial real estate,Jones Lang LaSalle,Derek Trulson,https://www.nyccfb.info/FTMSearch/Candidates/C...


In [20]:
real_estate_donors_for('Adams, Eric L').head(10).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 34770 to 183492
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   RECIPNAME       10 non-null     object 
 1   DATE            10 non-null     object 
 2   AMNT            10 non-null     float64
 3   cfb_name        10 non-null     object 
 4   ZIP             10 non-null     object 
 5   cfb_occupation  10 non-null     object 
 6   cfb_employer    10 non-null     object 
 7   name            10 non-null     object 
 8   cfb_link        10 non-null     object 
dtypes: float64(1), object(8)
memory usage: 800.0+ bytes


In [21]:
len(flagged_donations[flagged_donations['RECIPNAME'] == 'Adams, Eric L']['NAME'].unique())

319

In [22]:
pathlib.Path("donations").mkdir(parents=True, exist_ok=True)

# Saves two CSV files for candidate
#  candidate_top_realestate_donors.csv
#  canddiate_top_donors.csv
# see real_estate_donors_for() and all_top_donors_for()
for cand in ['Adams, Eric L', 'Donovan, Shaun', 'Garcia, Kathryn A', 'McGuire, Raymond J', 'Stringer, Scott M', 'Yang, Andrew']:
    path = f"donations/{candidate_last_names.get(cand)}_top_realestate_donors.csv"
    real_estate_donors_for(cand).head(50).to_csv(path, index=False)
    
    path = f"donations/{candidate_last_names.get(cand)}_top_donors.csv"
    all_top_donors_for(cand).head(250).to_csv(path, index=False)
    


In [23]:
# Top Real Estate Donors
largest_donors_per_candidate(flagged_donations_major_candidates, limit=3)

AMNT
RECIPNAME          NAME                          
Adams, Eric L      Boutross, Diana        $10,000
                   tamir, Elliot           $5,100
                   Verrone, Robert         $5,100
Donovan, Shaun     KATZ, JEFFREY           $5,100
                   Clark, Ric              $5,100
                   Milstein, Constance J   $5,100
Garcia, Kathryn A  Durst, Helena           $2,000
                   Kerr, Zachary           $2,000
                   Mosler, Wendy           $2,000
McGuire, Raymond J Colacino, Michael       $8,433
                   winter, benjamin        $5,100
                   Reffkin, Robert         $5,100
Morales, Dianne    Rosenthal, Richard        $500
                   Neveloff, Jay             $400
                   Fried, Daniel             $350
Stringer, Scott M  Klein, John S          $10,100
                   Quinlan, Robert         $7,900
                   Woo, Young              $5,100
Wiley, Maya D      Shubert, William        $2,000
                   Khan, Fazeel            $2,000
                   Shaw Clouse, Michelle   $1,200
Yang, Andrew       Shiah, Patricia         $2,250
                   Smeke, Alberto          $2,000
                   Dai, Shang              $2,000

In [24]:
def format_supported_candidates(names):
    out = set()
    
    for n in names:
        out.add(HumanName(n).last)
    
    return ", ".join(out)
    

# df = mayor_donations.groupby(["NAME"]).agg({"AMNT": 'sum', 'RECIPNAME': lambda series: list(series.unique()) }).sort_values(by="AMNT", ascending=False).head(50).reset_index()
# df.rename(columns={"NAME": "Donor Name", "AMNT": "Amount", "RECIPNAME": "Supported Candidates" }, inplace=True)
# df['Donor Name'] = df['Donor Name'].apply(to_person_name)
# df["Amount"] = df["Amount"].apply(lambda x: f"${humanize.intcomma(round(x))}" )
# df["Supported Candidates"] = df["Supported Candidates"].apply(format_supported_candidates)



In [25]:
def top_donors_for(df, head=20):
    top_donors = df.groupby(["NAME"]).agg({"AMNT": 'sum', 'RECIPNAME': lambda series: list(series.unique()) }).sort_values(by="AMNT", ascending=False).head(head).reset_index()
    top_donors.rename(columns={"NAME": "Donor Name", "AMNT": "Amount", "RECIPNAME": "Supported Candidates" }, inplace=True)
    top_donors['Donor Name'] = top_donors['Donor Name'].apply(to_person_name)
    top_donors["Amount"] = top_donors["Amount"].apply(lambda x: f"${humanize.intcomma(round(x))}" )
    top_donors["Supported Candidates"] = top_donors["Supported Candidates"].apply(format_supported_candidates)
    return top_donors


all_top_donors = top_donors_for(mayor_donations, head=50)
top_real_estate_donors = top_donors_for(flagged_donations, head=50)


In [26]:
top_real_estate_donors

,Donor Name,Amount,Supported Candidates
0,Winston Fisher,"$14,200","Yang, Donovan, Stringer, Adams"
1,Daniel Brodsky,"$12,200","Yang, Donovan, Stringer"
2,Derek Trulson,"$12,200","McGuire, Donovan, Adams"
3,John S Klein,"$10,100",Stringer
4,Diana Boutross,"$10,000",Adams
5,Kenneth Fisher,"$9,100","Yang, Stringer, Adams"
6,Michael Colacino,"$8,433",McGuire
7,Robert Quinlan,"$7,900",Stringer
8,Andrew Brooks,"$7,100","McGuire, Adams"
9,Robert Verrone,"$7,100","Yang, Adams"


In [27]:
all_top_donors

,Donor Name,Amount,Supported Candidates
0,Raymond J McGuire,"$1,000,000",McGuire
1,Shaun Donovan,"$19,100",Donovan
2,Anne Williams-Isom,"$17,300","Wiley, McGuire"
3,Ruth Blumenstein,"$15,300",Donovan
4,Allison Lutnick,"$15,300","Donovan, McGuire"
5,John Petry,"$14,200","Garcia, McGuire, Yang, Adams"
6,Winston Fisher,"$14,200","Yang, Donovan, Stringer, Adams"
7,James Tisch,"$14,200","Garcia, McGuire, Adams"
8,Steven Rattner,"$14,200","McGuire, Donovan, Stringer, Adams"
9,Judith Rubin,"$13,050","Yang, Stringer, Donovan, Morales, Garcia, McGuire"


In [28]:
# Top Employers Mayors
mayor_donations[["EMPNAME", "AMNT"]].groupby(["EMPNAME"]).agg(['sum', 'count']).sort_values(by=('AMNT', 'sum'), ascending=False).head(100)

AMNT       
                                       sum  count
EMPNAME                                          
Not Employed                    4957472.69  39042
Self Employed                   2982305.87  10165
Retired                          585065.19   1639
Paul Weiss                       206473.42    129
Citigroup                        159711.71    232
Homemaker                        118064.00    197
Skadden Arps                     101175.00    102
Self Employee                     85935.00    116
Wachtell, Lipton, Rosen & Katz    73825.00     57
Greenberg Traurig                 69275.00     35
None                              68661.46    114
Columbia University               54285.88    421
Google                            53283.68    406
City of New York                  50477.85    679
retired                           46738.31    117
Pershing Square Capital Manage    43200.00     11
NYU                               42834.59    361
Monroe College                    38430.00     37
Sullivan & Cromwell LLP           38430.00     37
Kirkland & Ellis LLP              37184.04     28
Lazard                            37000.00     10
Centerview Partners               34600.00      8
Skadden Arps Slate Meagher & F    33900.00     18
Cravath, Swaine & Moore LLP       31460.00     20
NYPD                              31330.81    277
Compass                           30121.21     78
Kirkland & Ellis                  29810.00     20
Student                           28760.10     92
NYC DOE                           26882.28    541
Fisher Brothers                   26455.00     11
CBRE                              26250.00     29
Morgan Stanley                    25115.21     63
Bank of America                   24682.42     94
PJT Partners                      24600.00     10
ViacomCBS                         24516.21     76
Mediaocean                        24161.00     46
Long Pond Capital                 23900.00      6
JLL                               23065.00     25
Willkie Farr & Gallagher LLP      22211.00     29
Standard Industries               21800.00      7
Cleary Gottlieb                   21600.00     15
Cleary Gottlieb Steen & Hamilt    21550.00     14
Greenberg Traurig, LLP            20700.00      5
Bloomberg                         20290.21     63
Fordham University                20153.42     66
DSNY                              19384.07    209
Joele Frank                       19001.75     13
CUNY                              18639.84    307
NYC                               18259.81    263
Shearman & Sterling LLP           18167.70     19
Simpson Thacher & Bartlett LLP    18005.00     18
Peloton                           17786.18     13
New York University               17746.31    241
Apple                             17413.05     78
King & Spalding LLP               17375.00     13
Sullivan & Cromwell               17320.00     21
Eminence Capital                  17300.00      4
Mastercard                        17120.00     37
King & Spalding                   17000.00     13
Third Point LLC                   16600.00     13
Douglas Elliman                   16575.00     35
Tashkent Supermarket              16000.00      7
Ehrenkranz Partners               15875.00      7
Fairstead                         15800.00      4
Estee Lauder Companies            15520.00      7
Corcoran                          15440.21     30
Facebook                          15402.25    124
Allen & Company LLC               15400.00      4
Greenberg Traurig LLP             15368.00      6
TG Holdings, LLC                  15300.00      3
Loews Corp                        15200.00      5
Rubenstein Communications         15150.00      3
Sessa Capital                     14950.00      7
Willett Advisors LLC              14200.00      4
The New School                    13978.51    124
Harvard Business School           13950.00      8
NYC Department of Education       13854.53    199
Newmark                           1385

In [29]:
# Top Employers Overall
contributions[["EMPNAME", "AMNT"]].groupby(["EMPNAME"]).agg(['sum', 'count']).sort_values(by=('AMNT', 'sum'), ascending=False).head(50)

AMNT       
                                       sum  count
EMPNAME                                          
Not Employed                    8419627.35  59128
Self Employed                   8335968.22  39845
MediaOcean                      6800000.00      6
Retired                         1384217.32   7453
Hess Corporation                1005100.00      4
Third Point LLC                  520125.00     19
Duquesne Family Office LLC       509100.00      4
Citadel                          503985.00     15
Daryl Roth Productions           501000.00      2
Tudor Investment Corp            500000.00      1
RFR Holding, LLC                 300000.00      3
Estee Lauder Companies           266680.00     19
Paul Weiss                       265804.42    303
Pershing Square Capital          250250.00      2
Homemaker                        242127.14    678
Eminence Capital                 219300.00      7
Citigroup                        180437.71    391
City of New York                 154396.44   2104
General Catalyst                 137200.00      4
Cantor Fitzgerald                131405.00     12
Kingdon Capital                  125000.00      2
NYC DOE                          120447.42   1959
retired                          114121.51    934
None                             112445.38    461
Skadden Arps                     111034.00    130
Columbia University              107180.10   1068
Monroe College                   105040.00    119
Invemed                          100000.00      1
SNF USA, Inc.                    100000.00      1
Henry Crown & Co.                100000.00      1
Google                            94978.37    827
Wachtell, Lipton, Rosen & Katz    88039.00     96
Self Employee                     85985.00    117
Blue Ridge Capital Ventures       82100.00      3
Greenberg Traurig                 81610.00    100
NYPD                              77653.73    878
Liontree LLC                      75000.00      1
G9 Ventures, LLC                  75000.00      1
NYU                               73842.59    746
RETIRED                           73733.08    972
NYC Council                       66259.34    790
NYCDOE                            65563.89   1001
NYC Department of Education       60814.10    763
Bank of America                   59704.63    235
Fisher Brothers                   58455.00     28
Student                           55025.31    578
Sullivan & Cromwell LLP           54050.00     85
New York City Council             54049.97    633
CUNY                              53131.13    900
54 Madison Partners               50500.00      2

In [30]:
sum_from_real_estate = flagged_donations_major_candidates[['RECIPNAME', 'AMNT']].groupby('RECIPNAME').sum()
sum_from_real_estate = sum_from_real_estate.sort_values(by='AMNT', ascending=False)
sum_from_real_estate['AMNT'] = sum_from_real_estate['AMNT'].apply(to_money)

sum_from_real_estate

,AMNT
RECIPNAME,
"Adams, Eric L","$290,668"
"McGuire, Raymond J","$286,856"
"Donovan, Shaun","$189,977"
"Stringer, Scott M","$153,006"
"Yang, Andrew","$115,179"
"Garcia, Kathryn A","$57,656"
"Wiley, Maya D","$13,527"
"Morales, Dianne","$3,236"


In [31]:
total_amount_recieved = mayor_donations[["RECIPNAME", "AMNT"]].groupby("RECIPNAME").sum().sort_values(by='RECIPNAME')
total_amount_recieved["Real Estate"] = flagged_donations_major_candidates[['RECIPNAME', 'AMNT']].groupby('RECIPNAME').sum().sort_values(by='RECIPNAME')
total_amount_recieved = total_amount_recieved.reset_index().sort_values(by='Real Estate', ascending=False)

total_amount_recieved["Total"]  = total_amount_recieved["AMNT"].apply(to_money)
total_amount_recieved["Real Estate"] = total_amount_recieved['Real Estate'].apply(to_money)
total_amount_recieved['Name'] = total_amount_recieved['RECIPNAME'].apply(to_person_name)

total_amount_recieved[['Name', 'Total', 'Real Estate']].style.hide_index()

Name,Total,Real Estate
Eric L Adams,"$4,739,061","$290,668"
Raymond J McGuire,"$10,093,818","$286,856"
Shaun Donovan,"$2,858,059","$189,977"
Scott M Stringer,"$2,839,137","$153,006"
Andrew Yang,"$3,600,747","$115,179"
Kathryn A Garcia,"$1,242,811","$57,656"
Maya D Wiley,"$1,598,117","$13,527"
Dianne Morales,"$860,549","$3,236"


In [32]:
# Total of all donations to all major candidates
to_money(flagged_donations_major_candidates['AMNT'].sum())

'$1,110,104'

In [33]:
flagged_donations_major_candidates['AMNT'].sum() / mayor_donations['AMNT'].sum() * 100

3.9885471996564004

In [34]:
# Save CSV of top 100 employers
top_employers = mayor_donations[["EMPNAME", "AMNT"]].copy()
top_employers['EMPNAME'] = top_employers['EMPNAME'].apply(employer_format)

top_employers = top_employers.groupby('EMPNAME').sum().sort_values('AMNT', ascending=False).reset_index()[['EMPNAME', 'AMNT']].sort_values(by='AMNT', ascending=False)
top_employers['Employer Name'] = top_employers['EMPNAME']
top_employers['Amount'] = top_employers['AMNT'].apply(to_money)

top_employers[['Employer Name', 'Amount']].head(100).to_csv('top_100_employers.csv', index=False)

In [35]:
employer_agg = mayor_donations[["RECIPNAME", "EMPNAME", "AMNT"]].copy().groupby(["RECIPNAME", "EMPNAME"]).agg(['sum', 'count']).sort_values(by=('AMNT', 'sum'), ascending=False)
employer_agg.head(50)

AMNT       
                                                          sum  count
RECIPNAME          EMPNAME                                          
McGuire, Raymond J Not Employed                    1640676.68   2288
Yang, Andrew       Not Employed                    1002820.93  11955
McGuire, Raymond J Self Employed                    946095.00   1040
Donovan, Shaun     Not Employed                     861399.15   1824
Adams, Eric L      Self Employed                    846381.00   1451
Stringer, Scott M  Not Employed                     486720.92   4354
Wiley, Maya D      Not Employed                     449755.00  10824
Donovan, Shaun     Self Employed                    334462.84    580
McGuire, Raymond J Retired                          306499.97    273
Garcia, Kathryn A  Not Employed                     271229.44   2331
Stringer, Scott M  Self Employed                    248975.50   1186
Yang, Andrew       Self Employed                    221548.79   1754
Morales, Dianne    Not Employed                     208498.57   5166
McGuire, Raymond J Paul Weiss                       198390.42     99
Wiley, Maya D      Self Employed                    190220.25   2292
McGuire, Raymond J Citigroup                        151267.71    167
Garcia, Kathryn A  Self Employed                    126161.75    816
Adams, Eric L      Retired                          109607.00    393
McGuire, Raymond J Skadden Arps                      96935.00     85
Stringer, Scott M  Retired                           96573.88    642
Adams, Eric L      Self Employee                     85935.00    116
Morales, Dianne    Self Employed                     68460.74   1046
Stringer, Scott M  Homemaker                         67195.00    108
McGuire, Raymond J Greenberg Traurig                 64550.00     18
                   None                              64133.46     49
                   Wachtell, Lipton, Rosen & Katz    58600.00     29
Adams, Eric L      Homemaker                         45639.00     82
Wiley, Maya D      Retired                           44677.63    277
McGuire, Raymond J Pershing Square Capital Manage    39200.00      9
                   Lazard                            36600.00      9
Adams, Eric L      Not Employed                      36372.00    300
McGuire, Raymond J Sullivan & Cromwell LLP           35050.00     21
                   Skadden Arps Slate Meagher & F    33800.00     17
Stringer, Scott M  Monroe College                    32400.00     23
McGuire, Raymond J Cravath, Swaine & Moore LLP       31100.00     15
                   Centerview Partners               30600.00      6
                   Kirkland & Ellis LLP              28925.00     20
Adams, Eric L      NYPD                              28775.00    220
McGuire, Raymond J Kirkland & Ellis                  26600.00     11
                   retired                           25868.15     19
                   PJT Partners                      24600.00     10
Donovan, Shaun     Mediaocean                        24161.00     46
McGuire, Raymond J Long Pond Capital                 23900.00      6
Donovan, Shaun     Retired                           22180.00     10
Garcia, Kathryn A  City of New York                  21665.65    246
McGuire, Raymond J Cleary Gottlieb Steen & Hamilt    21450.00     13
                   Compass                           21150.21     23
                   Greenberg Traurig, LLP            20700.00      5
                   Standard Industries               19800.00      6
                   Willkie Farr & Gallagher LLP      19400.00     14

In [36]:
def employer_donations(name):    
    return mayor_donations[mayor_donations["EMPNAME"].str.lower() == name.lower()]

def employer_donations_contains(name):    
    return mayor_donations[mayor_donations["EMPNAME"].str.lower().contains(name.lower)]

def candidates_supported(employer_name):
    recipients = employer_donations(employer_name)[["RECIPNAME", "AMNT"]].groupby('RECIPNAME').sum().reset_index().sort_values('AMNT', ascending=False).to_dict('records')
    return list(map(lambda x: f"{to_person_name(x['RECIPNAME'])} {to_money(x['AMNT'])}", recipients))

def employer_donations_summary(name, include_donors=False):
    d = { 'name': name }
    d['sum'] = round(employer_donations(name)['AMNT'].sum())
    d['recipients'] = candidates_supported(name)
    
    donors = list(map(to_person_name, employer_donations(name)["NAME"].unique()))
    
    d['donor_count'] = len(donors)
    d['donor_avg'] = to_money(d['sum'] / d['donor_count'])
    
    if include_donors:
        d['donors'] = donors
    
    return d
    

In [37]:
employer_donations_summary("NETFLIX")

{'name': 'NETFLIX',
 'sum': 12411,
 'recipients': ['Raymond J McGuire $10,350',
  'Shaun Donovan $1,275',
  'Andrew Yang $331',
  'Scott M Stringer $300',
  'Dianne Morales $100',
  'Maya D Wiley $55'],
 'donor_count': 17,
 'donor_avg': '$730'}

In [38]:
employer_donations_summary("Kirkland & Ellis", include_donors=True)

{'name': 'Kirkland & Ellis',
 'sum': 29810,
 'recipients': ['Raymond J McGuire $26,600',
  'Scott M Stringer $2,400',
  'Shaun Donovan $400',
  'Andrew Yang $300',
  'Kathryn A Garcia $100',
  'Maya D Wiley $10'],
 'donor_count': 18,
 'donor_avg': '$1,656',
 'donors': ['reginald brown',
  'Lauren Friedman',
  'Daniel Fuglestad',
  'Jacqueline Haberfeld',
  'Jonathan Henes',
  'Jia Li Huang',
  'Melissa Hutson',
  'Jason Kanner',
  'Michael Kim',
  'Peter Leung',
  'Song Lin',
  'Ramiro Ocasio',
  'Jen Perkins',
  'Edward Sassower',
  'Monica Shilling',
  'Eric Wedel',
  'Erica Williams',
  'Ali Zaidi']}

In [39]:
employer_donations_summary("Paul Weiss")

{'name': 'Paul Weiss',
 'sum': 206473,
 'recipients': ['Raymond J McGuire $198,390',
  'Andrew Yang $4,325',
  'Maya D Wiley $2,170',
  'Scott M Stringer $600',
  'Shaun Donovan $350',
  'Kathryn A Garcia $315',
  'Dianne Morales $273',
  'Eric L Adams $50'],
 'donor_count': 107,
 'donor_avg': '$1,930'}

In [40]:
employer_donations_summary("Google")

{'name': 'Google',
 'sum': 53484,
 'recipients': ['Andrew Yang $18,252',
  'Raymond J McGuire $12,065',
  'Kathryn A Garcia $7,810',
  'Dianne Morales $6,012',
  'Shaun Donovan $3,335',
  'Maya D Wiley $3,004',
  'Scott M Stringer $2,605',
  'Eric L Adams $400'],
 'donor_count': 240,
 'donor_avg': '$223'}

In [41]:
employer_donations_summary("Apple")

{'name': 'Apple',
 'sum': 17463,
 'recipients': ['Shaun Donovan $5,875',
  'Raymond J McGuire $5,350',
  'Maya D Wiley $2,405',
  'Andrew Yang $2,303',
  'Dianne Morales $1,354',
  'Eric L Adams $100',
  'Kathryn A Garcia $50',
  'Scott M Stringer $25'],
 'donor_count': 57,
 'donor_avg': '$306'}

In [42]:
employer_donations_summary("A&E Real Estate")

{'name': 'A&E Real Estate',
 'sum': 15100,
 'recipients': ['Raymond J McGuire $12,700',
  'Eric L Adams $2,000',
  'Kathryn A Garcia $400'],
 'donor_count': 5,
 'donor_avg': '$3,020'}

In [43]:
employer_donations_summary("Long Pond Capital")

{'name': 'Long Pond Capital',
 'sum': 23900,
 'recipients': ['Raymond J McGuire $23,900'],
 'donor_count': 6,
 'donor_avg': '$3,983'}

In [44]:
lobbyists = pd.read_csv("eLobbyist_data.csv", header=0)
lobbyist_orgs = lobbyists["LOBBYIST_NAME"].unique()
lobbyist_employees = set(itertools.chain(*map(lambda x: x.split(';'), lobbyists["LOBBYIST_EMPLOYEES"])))

In [45]:
lobbyist_employees_variations = list(map(lambda s: s.upper(), itertools.chain(*map(parse_name, lobbyist_employees))))
lobbyist_contributions = mayor_donations[mayor_donations['NAME'].str.upper().isin(lobbyist_employees_variations) | mayor_donations['OCCUPATION'].str.upper().eq('LOBBYIST')]
lobbyist_contributions

,ELECTION,OFFICECD,RECIPID,CANCLASS,RECIPNAME,COMMITTEE,FILING,SCHEDULE,PAGENO,SEQUENCENO,REFNO,DATE,REFUNDDATE,NAME,C_CODE,STRNO,STRNAME,APARTMENT,BOROUGHCD,CITY,STATE,ZIP,OCCUPATION,EMPNAME,EMPSTRNO,EMPSTRNAME,EMPCITY,EMPSTATE,AMNT,MATCHAMNT,PREVAMNT,PAY_METHOD,INTERMNO,INTERMNAME,INTSTRNO,INTSTRNM,INTAPTNO,INTCITY,INTST,INTZIP,INTEMPNAME,INTEMPSTNO,INTEMPSTNM,INTEMPCITY,INTEMPST,INTOCCUPA,PURPOSECD,EXEMPTCD,ADJTYPECD,RR_IND,SEG_IND,INT_C_CODE,realestate_littlesis,worst_landlord,works_in_realestate,does_lobbying
2073,2021,1,2303,P,"Donovan, Shaun",H,6,ABC,NaN,NaN,R0003122,9/13/2020,NaN,"Adams, Amelia",IND,NaN,NaN,NaN,M,New York,NY,10004,Consultant,Adams Advisors LLC,One,Battery Park Plaza,New York,NY,50.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False
2074,2021,1,2303,P,"Donovan, Shaun",H,7,ABC,NaN,NaN,R0014467,3/6/2021,NaN,"Adams, Amelia",IND,NaN,NaN,NaN,M,New York,NY,10004,Consultant,Adams Advisors LLC,One,Battery Park Plaza,New York,NY,50.0,0,50.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False
2075,2021,1,2303,P,"Donovan, Shaun",H,7,ABC,NaN,NaN,R0014962,3/9/2021,NaN,"Adams, Amelia",IND,NaN,NaN,NaN,M,New York,NY,10004,Consultant,Adams Advisors LLC,One,Battery Park Plaza,New York,NY,100.0,0,100.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False
2147,2021,11,2242,P,"Morales, Dianne",H,7,ABC,NaN,NaN,R0028213,3/10/2021,NaN,"Adams, Elizabeth",IND,NaN,NaN,NaN,K,Brooklyn,NY,11222,Legislative director,Nyc council,410,Atlantic avenue,Brooklyn,NY,40.0,40,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False
2148,2021,11,2242,P,"Morales, Dianne",H,8,ABC,NaN,NaN,R0046578,5/17/2021,NaN,"Adams, Elizabeth",IND,NaN,NaN,NaN,K,Brooklyn,NY,11222,Legislative director,Nyc council,410,Atlantic avenue,Brooklyn,NY,21.0,21,40.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364163,2021,11,2393,P,"Wiley, Maya D",H,8,ABC,NaN,NaN,R0070262,5/17/2021,NaN,"Youdelman, Sondra",IND,NaN,NaN,NaN,K,Brooklyn,NY,11238,Campaigns Director,People's Action,32,Clifton Place,Brooklyn,NY,50.0,50,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False
364492,2021,11,2393,P,"Wiley, Maya D",H,8,ABC,NaN,NaN,R0051865,4/16/2021,NaN,"Young, Patrick",IND,NaN,NaN,NaN,Q,East Elmhurst,NY,11369,Pastor,First Baptist Church,100-10,Astoria Blvd,East Elmhurst,NY,100.0,100,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False
365085,2021,11,2242,P,"Morales, Dianne",H,8,ABC,NaN,NaN,R0047765,5/17/2021,NaN,"Yuan, Yaya",IND,NaN,NaN,NaN,M,New York,NY,10034,Program Director,Apex for Youth,120,Walker St.,New York,NY,50.0,50,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False
365171,2021,11,1545,P,"Adams, Eric L",L,1,ABC,NaN,NaN,R0005372,7/12/2018,NaN,"Yuille, Antonia",IND,NaN,NaN,NaN,K,Brooklyn,NY,11213,Director Corporate Affairs,Con Edison,30,Flatbush Ave,Brooklyn,NY,50.0,0,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,False


In [46]:
num_unique_names = len(lobbyist_contributions['NAME'].unique())
total_amount = to_money(lobbyist_contributions['AMNT'].sum())

"{} lobbyists have given {} to mayoral candidates".format(num_unique_names, total_amount)

'734 lobbyists have given $319,369 to mayoral candidates'

In [47]:
largest_donors_per_candidate(lobbyist_contributions, limit=5)

AMNT
RECIPNAME          NAME                        
Adams, Eric L      Gray, William         $5,000
                   Ng, David             $2,500
                   Wachtel, Madeline     $2,000
                   McCormick, Maggie     $2,000
                   Mack, William         $2,000
Donovan, Shaun     Wilson, John          $5,100
                   Miller, Elizabeth     $2,000
                   Merrill, Thomas       $2,000
                   Smith, Michael        $1,000
                   Greenberger, Sharon   $1,000
Garcia, Kathryn A  Stern, Andrew         $1,150
                   Murphy, Michael       $1,100
                   Mansky, Adam          $1,000
                   Bystryn, Marcia       $1,000
                   Johnson, Jennifer       $750
McGuire, Raymond J Ford, Mary           $10,200
                   Klein, Michael        $5,100
                   Goldberg, Jeffrey     $5,100
                   Boxer, Leonard        $5,100
                   Williams, Steven      $5,100
Morales, Dianne    Little, Keith         $2,350
                   Arroyo, Maritza         $675
                   Davis, Gordon           $650
                   Watler, Christopher     $500
                   Jackson, Michelle       $450
Stringer, Scott M  Fisher, Kenneth       $5,100
                   Kelly, Mary           $3,000
                   Rock, Ron             $3,000
                   Levine, Randy         $3,000
                   Lespinasse, Patrick   $2,000
Wiley, Maya D      Rubin, Alan           $1,750
                   Johnson, Michael      $1,000
                   Baxter, Ryan            $925
                   Hinds, Janella          $750
                   Hirsh, Alison           $553
Yang, Andrew       Lee, Eric             $2,050
                   Miller, Richard       $2,000
                   Williams, Steven      $2,000
                   Silverman, Eric       $2,000
                   Fisher, Kenneth       $2,000

In [48]:
lobbyist_contributions_group = lobbyist_contributions[["RECIPNAME", "AMNT"]].groupby("RECIPNAME").sum().sort_values(by="AMNT", ascending=False)
lobbyist_contributions_group["AMNT"]  = lobbyist_contributions_group["AMNT"].apply(to_money)
lobbyist_contributions_group = lobbyist_contributions_group.rename_axis('Candidates').rename(columns={'AMNT': 'Amount'})
lobbyist_contributions_group

,Amount
Candidates,
"McGuire, Raymond J","$90,405"
"Adams, Eric L","$51,725"
"Stringer, Scott M","$50,155"
"Garcia, Kathryn A","$32,747"
"Donovan, Shaun","$28,363"
"Yang, Andrew","$26,559"
"Morales, Dianne","$19,852"
"Wiley, Maya D","$19,562"


In [49]:
lobbyist_contributions.sort_values(['RECIPNAME', 'DATE']).to_csv('flagged_lobbyist_donations.csv')

In [50]:
mayor_donations[mayor_donations['does_lobbying']]

,ELECTION,OFFICECD,RECIPID,CANCLASS,RECIPNAME,COMMITTEE,FILING,SCHEDULE,PAGENO,SEQUENCENO,REFNO,DATE,REFUNDDATE,NAME,C_CODE,STRNO,STRNAME,APARTMENT,BOROUGHCD,CITY,STATE,ZIP,OCCUPATION,EMPNAME,EMPSTRNO,EMPSTRNAME,EMPCITY,EMPSTATE,AMNT,MATCHAMNT,PREVAMNT,PAY_METHOD,INTERMNO,INTERMNAME,INTSTRNO,INTSTRNM,INTAPTNO,INTCITY,INTST,INTZIP,INTEMPNAME,INTEMPSTNO,INTEMPSTNM,INTEMPCITY,INTEMPST,INTOCCUPA,PURPOSECD,EXEMPTCD,ADJTYPECD,RR_IND,SEG_IND,INT_C_CODE,realestate_littlesis,worst_landlord,works_in_realestate,does_lobbying
10703,2021,11,1545,P,"Adams, Eric L",L,2,ABC,NaN,NaN,R0009356,1/9/2019,NaN,"Andrews, Emanuel",IND,NaN,NaN,NaN,Z,Esperance,NY,12066,Lobbyist,Self Employed,111,Washington Avenue,Albany,NY,500.00,0,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,True
50882,2021,11,148,P,"Stringer, Scott M",R,3,ABC,NaN,NaN,R0038915,7/5/2019,NaN,"Cassidy, Michael",IND,NaN,NaN,NaN,Z,Albany,NY,12208,Lobbyist,Brown Weinraub,50,State St,Albany,NY,400.00,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,True
72065,2021,11,2451,P,"Garcia, Kathryn A",H,8,ABC,NaN,NaN,R0027390,5/16/2021,NaN,"Cudahy, Kathleen",IND,NaN,NaN,NaN,M,New York,NY,10128,Lobbyist,Connelly McLaughlin &Woloz,233,Broadway,New York,NY,400.00,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,True
74461,2021,11,1545,P,"Adams, Eric L",L,3,ABC,NaN,NaN,R0010638,5/9/2019,NaN,"Daly, Samara",IND,NaN,NaN,NaN,K,Brooklyn,NY,11231,Lobbyist,Self Employed,16,Court Street,Brooklyn,NY,175.00,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,True
74462,2021,11,1545,P,"Adams, Eric L",L,7,ABC,NaN,NaN,R0030027,2/4/2021,NaN,"Daly, Samara",IND,NaN,NaN,NaN,K,Brooklyn,NY,11231,Lobbyist,Self Employed,16,Court Street,Brooklyn,NY,100.00,0,175.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,True
74489,2021,11,2451,P,"Garcia, Kathryn A",H,6,ABC,NaN,NaN,R0005238,1/10/2021,NaN,"Daly, Samara",IND,NaN,NaN,NaN,K,Brooklyn,NY,11231,Lobbyist,Bolton St. Johns,249,President Street,Brooklyn,NY,175.00,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,True
74490,2021,11,2451,P,"Garcia, Kathryn A",H,7,ABC,NaN,NaN,R0010877,3/3/2021,NaN,"Daly, Samara",IND,NaN,NaN,NaN,K,Brooklyn,NY,11231,Lobbyist,Bolton St. Johns,249,President Street,Brooklyn,NY,250.00,0,175.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,True
74492,2021,1,2470,NP,"McGuire, Raymond J",H,6,ABC,NaN,NaN,R0006385,12/15/2020,NaN,"Daly, Samara",IND,NaN,NaN,NaN,Z,Kingston,NY,12401,Lobbyist,Bolton St. Johns,7,World Trade Ctr,New York,NY,360.00,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,True
74493,2021,1,2470,NP,"McGuire, Raymond J",H,7,ABC,NaN,NaN,R0015101,1/28/2021,NaN,"Daly, Samara",IND,NaN,NaN,NaN,Z,Kingston,NY,12401,Lobbyist,Bolton St. Johns,7,World Trade Ctr,New York,NY,250.00,0,360.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,True
74495,2021,11,2546,P,"Yang, Andrew",H,7,ABC,NaN,NaN,R0049477,3/10/2021,NaN,"daly, samara",IND,NaN,NaN,NaN,K,Brooklyn,NY,11231,Lobbyist,Bolton St Johns,249,President Street,Brooklyn,NY,50.00,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,False,False,False,True
